# 기존 데이터셋 정제

In [4]:
file_path = "../data/origin/multi_news_train.tgt"
with open(file_path, 'r', encoding='utf-8') as f:
    target = f.readlines()
file_path = "../data/origin/multi_news_train.src.cleaned"
with open(file_path, 'r', encoding='utf-8') as f:
    source = f.readlines()

In [10]:
print(source[2126])

Karma's a brick. An attempted burglary in Suitland, Maryland, went all wrong when the suspect's brick ricocheted off bulletproof glass and into his face. Prince George's County police released surveillance video from the scene. (Published Monday, Oct. 1, 2018) NEWLINE_CHAR NEWLINE_CHAR An attempted burglary in Suitland, Maryland, went all wrong when the suspect's brick ricocheted off bulletproof glass and smashed into his head. NEWLINE_CHAR NEWLINE_CHAR Police are calling him the "Bad Luck Bandit." NEWLINE_CHAR NEWLINE_CHAR The suspect shattered the front glass to get inside a waiting area of a takeout restaurant, police said. Surveillance video shows him then trying to break through a window to a back room. NEWLINE_CHAR NEWLINE_CHAR The suspect lobbed the brick once, then twice, but the bulletproof glass resisted. He wound up his pitch for a third attempt and flung the brick to the window. It bounced back. NEWLINE_CHAR NEWLINE_CHAR The suspect flinched, but it was too late. NEWLINE_CH

In [29]:
from nltk.tokenize import sent_tokenize
def split_into_sentences(text):
    sentences = sent_tokenize(text)
    return sentences


In [30]:
all_result = []
for source_line, target_line in zip(source, target):
    result = {}
    all_sent = ""
    sentences = re.split(r'NEWLINE_CHAR|\|\|\|\|\|', source_line)
    for sentence in sentences:
        if sentence == " ":
            continue
        all_sent = all_sent + "\n" + sentence.strip()
    all_sent = all_sent.strip()
    
    result["document"] = all_sent
    result["summary"] = target_line.replace("– ", "")
    all_result.append(result)


In [31]:
print(all_result[0]["summary"])

The Da Vinci Code has sold so many copies—that would be at least 80 million—that it's bound to turn up in book donation piles. But at one charity shop in the UK, it's been donated so heavily that the shop has posted a sign propped up on a tower of Da Vinci Code copies that reads: "You could give us another Da Vinci Code... but we would rather have your vinyl!" The manager of the Oxfam shop in Swansea tells the Telegraph that people are laughing and taking pictures of the sizable display: "I would say that we get one copy of the book every day." He says people buy them "occasionally," but with vinyl sales up 25% in the past year, they'd rather take records. Dan Brown's book isn't the only one that shops like Oxfam struggle to re-sell. Last year, Oxfam was hit with a large and steady supply of Fifty Shades of Grey, and it similarly begged donors: "Please—no more." But Brown has a particular kind of staying power. The Da Vinci Code was published in 2003, and within six years Brown had boo

In [33]:
import json
file_path = "../data/multi-news_dev.json"
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

# train 데이터 만들기

In [9]:
from tqdm import tqdm

def create_example(all_data):
    all_result = []
    for data in tqdm(all_data):
        # data_id = data["id"]
        summary = data["summary"]
        context = data["document"].split("\n")
        total_sentence_number = 1
        sentences = ""
        for sent in context:
            sentence = "[{}] {}".format(total_sentence_number, sent) + "\n"
            sentences = sentences + sentence
            total_sentence_number += 1

        
        sentence = sentence.rstrip("\n")
        instruction = "<|mrc|>False\n<|summary|>True"
        prompt = (
            "**Document**\n{}".format(sentences)
        )

        response = "**Answer**\n**Summary**\n{}".format(summary)
        # response = "**Answer**: {}\n**Supporting Sentences**: {}".format(answer, supporting_sentence)
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt},
            # {"role": "assistant", "content": response},
        ]
        result = {}
        result["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response},
        ]
        result["all_text"] = tokenizer.apply_chat_template(messages, tokenize=False)
        result["label"] = response
        all_result.append(result)
        supporting_sentence = ""
        # print(model_answer)
        # if len(all_result) ==1000:
        #     break
    return all_result
    

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_path = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer.padding_side = 'left'

/home/rbqlsquf2/workspace/XAI_rationale-inference-LLM/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.36s/it]


In [3]:
new_special_tokens = {"additional_special_tokens": ["<|mrc|>", "<|summary|>"]}

In [4]:
tokenizer.add_special_tokens(new_special_tokens)

2

In [5]:
model.resize_token_embeddings(len(tokenizer))

Embedding(151667, 2048)

In [10]:
import json
file_path = "../data/multi-news_dev.json"
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
input_data = create_example(data)

100%|██████████| 5622/5622 [00:00<00:00, 10296.73it/s]


In [27]:
with open("../data/qwen_multi_news_train_chat_tmp.json", "w", encoding="utf-8") as f:
    json.dump(input_data, f, ensure_ascii=False, indent=4)

In [10]:
from tqdm import tqdm
all_len = []
all_result = []

for input_data_ in tqdm(input_data):
    text = input_data_["text"]
    count = len(tokenizer(text)["input_ids"])
    if count <= 2048:
        # data["text"] = data["text"]
        all_result.append(input_data_)
    all_len.append(count)

with open("../data/qwen_multi_news_test_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

100%|██████████| 5622/5622 [00:24<00:00, 228.00it/s]


In [34]:
print(all_result[0])

{'text': '<|im_start|>system\n<|mrc|>False\n<|summary|>True<|im_end|>\n<|im_start|>user\n**Document**\n[1] National Archives\n[2] Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs.\n[3] A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%.\n[4] Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you’re here, why don’t you sign up to follow us on Twitter.\n[5] Enjoy the show.\n[

In [12]:
all_len = []
all_result = []
over_num = 0
for input_data_ in tqdm(input_data):
    text = input_data_["all_text"]
    count = len(tokenizer(text)["input_ids"])
    if count <= 2048:
        all_result.append(input_data_)
    else:
        over_len = count - 2048
        input_data_['text'] = input_data_['text'][:over_len]
        over_num +=1
    # all_len.append(len(tokenizer(text)["input_ids"]))
print(over_num)

100%|██████████| 5622/5622 [00:28<00:00, 198.78it/s]

3005


In [13]:
with open("data/qwen_cnn_test_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'data/qwen_cnn_test_data.json'

In [38]:
for result in all_result:
    result["label"] = "assistant\n" + result["label"]
    

In [39]:
with open("data/qwen_dev_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [14]:
from tqdm import tqdm
all_len = []
all_result = []

for data, input_data_ in tqdm(zip(dev_data, input_data)):
    text = input_data_["text"]
    if len(tokenizer(text)["input_ids"]) <= 2048:
        # data["text"] = data["text"]
        all_result.append(data)
    # all_len.append(len(tokenizer(text)["input_ids"]))

7405it [00:17, 426.88it/s]


In [16]:
with open("data/teddst_dev.json", "w", encoding="utf-8") as f:
    json.dump(input_data, f, ensure_ascii=False, indent=4)

In [7]:
print(input_data[0]["label"])

**Answer**: yes
**Supporting Sentences**: [4] Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.
[17] Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.



In [16]:
count = len(list(filter(lambda x: x < 2048, all_len)))
print(count)

6539
